In [ ]:
import tensorflow.keras as keras

from neural_collaborative_filtering.Dataset import Dataset

In [ ]:
keras.__version__

In [ ]:
# 논문저자 코드 참고

In [ ]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    num_items = train.shape[1]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [ ]:
dataset = Dataset("./neural_collaborative_filtering/Data/ml-1m")
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

In [ ]:
epochs = 20
batch_size = 256
learning_rate = 0.001
num_negatives = 4

# for gmf
latent_dim = 8

# for mlp
embedding_size = 64

# regs = [0, 0]

# 모델 확인용.. data를 실제로 넣어서 학습할때는 없애야함.
# user_num = 100
# item_num = 100
user_num = train.shape[0]
item_num = train.shape[1]

In [ ]:
# gmf
# user, item을 동일한 size의 latent space로.. embedding..후 element wise product..
user_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_gmf")
item_input_gmf = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_gmf")


user_embedding_gmf = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = latent_dim,
                                            name = "user_embedding_gmf",
#                                             embeddings_initializer = "uniform",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_gmf = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = latent_dim,
                                            name = "item_embedding_gmf",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_gmf = keras.layers.Flatten()(user_embedding_gmf(user_input_gmf))
item_latent_gmf = keras.layers.Flatten()(item_embedding_gmf(item_input_gmf))

element_wise_product = keras.layers.multiply([user_latent_gmf, item_latent_gmf])

prediction_gmf = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction_gmf")(element_wise_product)

model_gmf = keras.models.Model(inputs = [user_input_gmf, item_input_gmf],
                               outputs = prediction_gmf)

In [ ]:
model_gmf.summary()

In [ ]:
model_gmf.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = "binary_crossentropy")

In [ ]:
user_input, item_input, labels = get_train_instances(train, num_negatives)


hist = model_gmf.fit([np.array(user_input), np.array(item_input)],
                     np.array(labels),
                     batch_size = batch_size,
                     epochs = epochs,
                     verbose = 1,
                     shuffle = True)

In [ ]:
model_gmf.save_weights(filepath = "./model/model_gmf.h5")